# Simple Q&A System 

#### Cristobal Donoso Oliva
###### May 2018

In [630]:
import urllib2
from urllib2 import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
from util import *
import nltk
import numpy as np
from sklearn.cluster import DBSCAN,KMeans

In [631]:
print 'Ingrese busqueda...'
question = raw_input()
questions = re.split(r'[^\w\s\á\ó\í\é\ú]', question, re.U|re.I)
quest = ' '
for q in questions:
    if q != '':
        quest += q
formal_question = quest.replace(' ','+')

Ingrese busqueda...
Quien fue John Lennonn?


In [632]:
print '\nBuscando',quest,'...\n'
text = nltk.tokenize.word_tokenize(quest)
my_url = 'https://www.google.ru/search?q='+formal_question
print my_url


Buscando  Quien fue John Lennonn ...

https://www.google.ru/search?q=+Quien+fue+John+Lennonn


In [633]:
req = urllib2.Request(my_url, headers={'User-Agent' : "Magic Browser"}) 
con = urllib2.urlopen(req)
page_html = con.read()

page_soup = soup(page_html, "html.parser")

containers = page_soup.findAll("div", {'class':'g'})

answers = []
for container in containers:
    if 'href' in str(container):
        link = container.a['href'].split('&sa=')[0].split('/url?q=')
        if (len(link)>1) and (str(link).find("youtube") == -1):
            if text[0].lower() == 'donde':
                answer = get_info_where(link[1])
            else:
                answer = get_info_who(link[1])
            answers.append(answer)
print 'Ready!'

Reading  https://es.wikipedia.org/wiki/John_Lennon
-----------------------------------------------------------------------
Reading  https://es.wikipedia.org/wiki/John_Lennon
-----------------------------------------------------------------------
Reading  https://www.biografiasyvidas.com/biografia/l/lennon.htm
-----------------------------------------------------------------------
Reading  https://historia-biografia.com/john-lennon/
-----------------------------------------------------------------------
Reading  https://www.buscabiografias.com/biografia/verDetalle/1902/John%2520Lennon
-----------------------------------------------------------------------
Reading  https://www.todomusica.org/john_lennon/
-----------------------------------------------------------------------
Reading  https://www.quien.net/john-lennon.php
-----------------------------------------------------------------------
Reading  https://www.infobae.com/america/cultura-america/2017/12/08/como-fue-el-ultimo-dia-en-la-

In [634]:
word_chain = []
for answer in answers:
    if answer != None and answer != []:
        for sentence in answer:
            s = sentence.split()
            for word in s:
                w = word
                word_ = re.split(r'[\.|,|\;|\:]', w, re.U|re.I)[0]
                word_chain.append(word_.lower())

In [635]:
unique_words = list(set(word_chain))

In [636]:
X2 = []
sentences = []
for answer in answers:
    if answer != None:
        for sentence in answer:
            vector_ngram = np.zeros(len(unique_words))
            splitted = sentence.split()
            sentences.append(sentence)
            for word in splitted:
                w = word
                word_ = re.split(r'[\.|,|\;|\:]', w, re.U|re.I)[0]
                index = list(unique_words).index(word_.lower())
                vector_ngram[index] = 1
            X2.append(vector_ngram)

In [637]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X2)
labels = kmeans.labels_
labels_u = np.unique(labels )

In [638]:
from sklearn.cluster import AgglomerativeClustering
model = AgglomerativeClustering(linkage='complete',
                                affinity='cosine',
                                n_clusters=4).fit(X2)
labels = kmeans.labels_
labels_u = np.unique(labels )

In [639]:
all_clusters = []
for k in range(0,len(labels_u)):
    print'label:',k
    partial_cluster = []
    for i,(label) in enumerate(labels):
        if label == k:
            partial_cluster.append(sentences[i])
            print sentences[i]
    all_clusters.append(partial_cluster)        
    print '----'

label: 0
Fue asesinado tres semanas después de su lanzamiento.
Fue asesinado tres semanas después de su lanzamiento.
fue asesinado por un perturbado poco después de presentar su último trabajo, Double Fantasy.
fue su madre, Julia Stanley, la que desapareció, dejando el niño al cuidado de una hermana suya llamada Mary.
fue un músico y activista inglés, reconocido por ser miembro de la banda The Beatles.
es para violencia.
fue un desertor en la II Guerra Mundial y escapó como tripulante de un barco hasta el norte de África.
fue a vivir con otro hombre, con quien tuvo otro bebé, procuraba visitar a su hijo mayor a menudo para mantener cierto contacto.
fue sin lugar a dudas la agrupación de rock más destacada de la historia de la música y en ello por supuesto sus integrantes fueron clave.
----
label: 1
fue un artista, músico multiinstrumentista, poeta, dibujante, actor, pacifista y compositor británico que saltó a la fama como uno de los miembros fundadores de The Beatles, reconocida como 

In [640]:
final_answer = []
for sentence in all_clusters:
    if sentence[0] not in final_answer:
        final_answer.append(sentence[0])
                

In [697]:
if text[0].lower() == 'donde':
    df = pd.read_csv('paises.csv')
    countries = df['Pais'].str.lower()
    a = np.array(countries)
else:
    df = pd.read_csv('profesiones.csv')
    profesions = df['Profesion'].str.lower()
    a = np.array(profesions)

In [670]:
app = []
for i,(answer) in enumerate(final_answer[0:5]):
    item = str(i+1)+')'
    print item,answer
    answer_splitted = answer.split()
    count = 0
    for word in answer_splitted:
        word_ = re.split(r'[\.|,|\;|\:]', word, re.U|re.I)[0]
        if word_ in a:
            print word_
            count+=1
    app.append(count)

1) Fue asesinado tres semanas después de su lanzamiento.
2) fue un artista, músico multiinstrumentista, poeta, dibujante, actor, pacifista y compositor británico que saltó a la fama como uno de los miembros fundadores de The Beatles, reconocida como la banda más comercialmente exitosa y críticamente aclamada en la historia de la música popular.
artista
dibujante
compositor
3) es responsable de veinticinco sencillos número uno en el Billboard Hot 100.
4) Es difícil imaginar cómo sería hoy este artista polifacético, que tanto cambió a lo largo de su vida, si un fan perturbado como Mark David Chapman no le hubiera disparado a sangre fría junto al edificio Dakota de Nueva York, donde el músico vivía con Yoko Ono y su hijo Sean.
artista


/opt/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:9: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  if __name__ == '__main__':


In [693]:
app   

[0, 3, 0, 1]

In [695]:
scores = pd.DataFrame(final_answer[0:4])
col = ['answer']
scores.columns = col
scores['rank'] = app
scores.sort_values('rank', ascending=False)

,answer,rank
1,"fue un artista, músico multiinstrumentista, po...",3
3,Es difícil imaginar cómo sería hoy este artist...,1
0,Fue asesinado tres semanas después de su lanza...,0
2,es responsable de veinticinco sencillos número...,0
